In [1]:

import json
from data_preparation import DataHandling 



In [2]:

config_file = 'config.json'

with open(config_file, 'r') as f:
    config = json.load(f)
ga_data_dir = config["ga_data_dir"]
fdg_data_dir = config["fdg_data_dir"]
log_dir = config["log_dir"]
output_dir = config["output_dir"]


In [3]:

data_handler = DataHandling(ga_data_dir, train_mode="NAC", target_mode="ADCM")

train_files = data_handler.get_data_split('train')
val_files = data_handler.get_data_split('val')
test_files = data_handler.get_data_split('test')
test_files

from data_preparation import LoaderFactory
loader_factory = LoaderFactory(
    train_files=train_files,
    val_files=val_files,
    test_files=test_files,
    patch_size = [168, 168, 16],
    spacing = [4.07, 4.07, 3.00],
    spatial_size = (168, 168, 320),
    normalize=True
    )

# Get the DataLoader for each dataset type
train_loader = loader_factory.get_loader('train', batch_size=4, num_workers=2, shuffle=True)
val_loader = loader_factory.get_loader('val', batch_size=1, num_workers=2, shuffle=False)
test_loader = loader_factory.get_loader('test', batch_size=1, num_workers=2, shuffle=False)



Loading dataset: 100%|██████████| 131/131 [01:10<00:00,  1.85it/s]


In [4]:
from model_training import TrainingLogger, DecayLR


starting_epoch = 0
decay_epoch = 4
learning_rate = 0.001




In [5]:

import os
import torch
from model_maker import get_network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = get_network(patch_size = [168, 168, 16], spacing = [4.07, 4.07, 3.00])
# model.load_state_dict(torch.load('/students/2023-2024/master/Shahpouri/LOG/model_3_18_22_10.pth'))

model = model.to(device)


loss_function = torch.nn.MSELoss()

print('Defining optimizer...')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.5, 0.999))

max_epochs = 20
best_metric = float('inf')
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
train_losses = []
val_losses = []

# Define scheduler
print('Defining scheduler...')
lr_lambda = DecayLR(epochs=max_epochs, offset=0, decay_epochs=decay_epoch).step
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)




DyUnet is set:
Kernel size:  [[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]
Strides:  [[1, 1, 1], [2, 2, 2], [2, 2, 2], [2, 2, 1]]
Defining optimizer...
Defining scheduler...


In [7]:
from model_training import ModelTrainer
    
trainer = ModelTrainer(model, train_loader, val_loader, optimizer, loss_function, scheduler, max_epochs,log_dir, device)
# trainer.log()
trainer.train()




----------
epoch 1/20
1/32, train_loss: 1.9227
2/32, train_loss: 7.9976
3/32, train_loss: 1.5333
4/32, train_loss: 2.0363
5/32, train_loss: 1.0347
